[View in Colaboratory](https://colab.research.google.com/github/RayleighKim/M11_Intro_To_Unsuerpvised_Learning/blob/master/11_1_Intro_to_Unsupervised_Learning_I.ipynb)

# Intro to Unsupervised Learning

## Text Retrieval Problem

### Your name :     

### Data Link :
* [DBpedia](https://wiki.dbpedia.org/)
* [Dataset](https://media.githubusercontent.com/media/RayleighKim/Example_datasets/master/People_wiki.csv)

#### 실습목표 I<br>
1. Bag of Words, TF-IDF를 이해한다.
2. Scikit-Learn을 이용하여 Retrieval을 할 줄 안다.

#### 실습목표 II<br>
1. 문제의 구조를 정교화 하기 위해 발전한 과정의 흐름을 안다.
---------------
Rayleigh Kim @ D:plus


In [0]:
'''
라이브러리들을 불러오자.
'''
import numpy as np
import pandas as pd
'''
사이킷런의 라이브러리들을 미리 불러두는 것도 좋지만,
세부적인 메쏘드가 너무 많아서 필요한것만 불러오도록 할 것이다.
'''


## Load and Prepare the data

아래와 같은 방식으로, 깃에 올려둔 데이터 파일을 불러올 수도 있다.

단 용량이 꽤 큰 파일을 사용했더니 Github으로 부터 '너 돈 내고 쓰지 않을래..?' 라는 메일이 왔다!

그러니, 실습시에는 파일 업로드를 이용하자

In [0]:
# data_path = 'https://media.githubusercontent.com/media/RayleighKim/Example_datasets/master/People_wiki.csv'
# people = pd.read_csv(data_path)

In [0]:
# '''
# People_wiki.csv 를 업로드 하려하면 끔찍하게 느리다.
# '''
# from google.colab import files
# uploaded = files.upload()

In [0]:
from google.colab import drive
drive.mount('./gdrive')

In [0]:
cd gdrive/My Drive/ML_first

In [0]:
ls

In [0]:
import zipfile as zf
zf = zf.ZipFile('People_wiki.zip')
people = pd.read_csv(zf.open('People_wiki.csv'))

In [0]:
len(people)

In [0]:
people.head()

우리는 URL은 안쓸꺼니까! 날려버리자.

왜인지 URI로 되어 있다.

In [0]:
if 'URI' in list(people):
    people.drop(labels=['URI'], axis=1, inplace=True)
people.head()

## Explore the dataset

엘튼 존! 조지 클루니!

In [0]:
elton = people.loc[people['name']=='Elton John']
elton

In [0]:
list(elton['text'])

In [0]:
george = people.loc[people['name'].str.contains('George')]
george['name']

In [0]:
clooney = george.loc[george['name'].str.contains('Clooney')]
clooney

In [0]:
list(clooney['text'])

## Bag of Words Simple ver

텍스트 문서들은 있다.<br>

텍스트들을 단어 등장여부를 이용한 binary vector로 만들자.

In [0]:
'''
Library Loading
'''
from collections import Counter, OrderedDict
import sklearn.feature_extraction.text as text

### 문장 하나를 사전으로 만들어 보자.

In [0]:
def Ordered_words(id = 0):
    sample_string = people['text'][id]
    string_splitted = sample_string.split()
    word_dict = dict(Counter(string_splitted))
    word_dict_list = [(k,v) for k, v in word_dict.items()]
    word_dict_list.sort(key = lambda x:x[1], reverse=True)
    print('name : ',people['name'][id])
    print(sample_string)
    print(word_dict_list)
    

In [0]:
'''
Ordered_words(n)
n 을 바꾸어 가며 확인해보자.
'''

Ordered_words(3)

### 모든 텍스트를 이용하여, 존재하는 단어들의 사전을 만들어 보자!

In [0]:
word_bin = text.CountVectorizer(binary = True)

In [0]:
X_simple = word_bin.fit_transform(people['text'])

In [0]:
type(X_simple)

In [0]:
print(len(word_bin.vocabulary_))
word_bin.vocabulary_

### 문장 하나를 벡터로 만들어 보자!

 모든 문장을 이용하여 단어 사전을 만들어 두었다!
 
 이제, 문장 하나를 단어 존재 여부를 이용하여 벡터로 만들 수 있다.

In [0]:
def simple_BOW_vector(id=0):
    sample_array = word_bin.transform([people['text'][id]]).toarray()[0]
    print('array 사이즈 : ', len(sample_array))
    print('등장한 단어의 개수 : ', sum(sample_array==1))
    print(sample_array)

In [0]:
simple_BOW_vector(4)

### 문장들을 벡터로 만들어서 데이터 프레임에 붙일 수 있을까? -> 붙이지 말자!

정말 궁금한 사람들은 아래의 코드를 실행해보자.

메모리가 터지는 것을 관찰할 수 있다!

```
word_bin.transform(people['text'].toarray())
```

위에서 아래와 같은 코드를 사용했었다.

```
X_simple = word_bin.fit_transform(people['text'])
```

X_simple는 scipy.sparse.csr.csr_matrix 라고 위에서 확인했었는데,  사실 X가 우리가 원하는 그 메트릭스다!

다만, 사이즈가 54만이 넘어가는 벡터에서 고작 단어 몇백개의 위치만 1이 되는 array를 그대로 생성하는 것은 메모리가 괴로워하기 때문에, 우리는 잘 압축된 메트릭스를 대신하여 사용한다.

아래의 셀을 실행해보면 어떻게 된 것인지 감을 잡을 수 있다!

In [0]:
people.head(1)

In [0]:
simple_BOW_vector(0)

In [0]:
X_simple[0,:].todense()

In [0]:
X_simple[0,:].todense().sum()

In [0]:
print(X_simple[0,:])

### 우리는 sparse_matrix를 이용하여 그대로 nearest neighbors 도 해보고 클러스터링도 해볼 것이다!

NearestNeighbor search를 해보자!

In [0]:
from sklearn.neighbors import NearestNeighbors

In [0]:
nbrs_simple = NearestNeighbors(n_neighbors=6, metric='cosine').fit(X_simple)

In [0]:
def nearest_search(name, nbrs = nbrs_simple, X = X_simple):
    queried_id = people.index[people['name']==name]
    queried_array = X[queried_id, :]
    distances, indices = nbrs.kneighbors(queried_array)
    temp_df = people.loc[indices.tolist()[0]]
    temp_df['rank'] = [0,1,2,3,4,5]
    distances = [ '{:.6f}'.format(elem) for elem in distances.tolist()[0]]
    temp_df['distances'] = distances
#     print(temp_df)
    return(temp_df)
    

In [0]:
nearest_search(name = 'Angelina Jolie', nbrs = nbrs_simple, X = X_simple).head(3)

In [0]:
nearest_search(name = 'Victoria Beckham', nbrs = nbrs_simple, X = X_simple).head(3)

## Bag of Words Count ver

이제는 Bag of Words를 단어 수로 구현해보자!

In [0]:
'''
Library Loading
'''

import sklearn.feature_extraction.text as text

위에서는 아래와 같은 코드를 사용했다
```
word_bin = text.CountVectorizer(binary = True)
```

binary = True 가 되면, 단어의 등장 여부를 가지고 vector를 만들지만,

binary = False로 해두면, 단어의 등장 빈도를 이용하여 vector를 만든다.

In [0]:
word_count = text.CountVectorizer(binary = False)
X_count = word_count.fit_transform(people['text'])

In [0]:
people.head(1)

In [0]:
simple_BOW_vector(0)

In [0]:
X_count[0,:].todense()

In [0]:
print('총 단어 등장 수 :', X_count[0,:].todense().sum())
print('등장한 단어의 수 :', X_simple[0,:].todense().sum())

Bag of Words 가  등장여부에서 단어횟수로 더 정교화 되었다!

Nearest Neighbor search해보자!

In [0]:
from sklearn.neighbors import NearestNeighbors

In [0]:
nbrs_count = NearestNeighbors(n_neighbors=6, metric='cosine').fit(X_count)

In [0]:
nearest_search(name='Angelina Jolie', nbrs = nbrs_count, X = X_count)

In [0]:
nearest_search(name='Angelina Jolie', nbrs = nbrs_simple, X = X_simple)

In [0]:
nearest_search(name='Victoria Beckham', nbrs = nbrs_count, X = X_count)

In [0]:
nearest_search(name='Victoria Beckham', nbrs = nbrs_simple, X = X_simple)

## 이제는 TF-IDF 다!

다시 한 번 TF와 IDF를 복습하고 오자

In [0]:
'''
Library Loading
'''

import sklearn.feature_extraction.text as text

이 [문서](https://stackoverflow.com/questions/27697766/understanding-min-df-and-max-df-in-scikit-countvectorizer) 를 참고하면 max_df와 min_df 에 대한 이해를 할 수 있다

In [0]:
word_tfidf = text.TfidfVectorizer(max_df = 0.5)
X_tfidf = word_tfidf.fit_transform(people['text'])

잠시 idf 를 감상하자!

In [0]:
idf = word_tfidf.idf_

In [0]:
'''
단어 인덱스를 보자
'''
print(len(word_tfidf.vocabulary_))
word_tfidf.vocabulary_


In [0]:
'''
scikit learn은 기본적으로 idf에 1을 더해두었다.
max_df = 0.5 를 둘 경우, 총 문서의 50% 이상의 문서들에 출현하는 단어는
다 제외해 버린다!

당연히 the는 에러가 남
'''
id = word_tfidf.vocabulary_['the']
idf[id]

In [0]:
id = word_tfidf.vocabulary_['george']
idf[id]

In [0]:
id = word_tfidf.vocabulary_['metal']
idf[id]

0번 텍스트에 대해서 서로의 차이점을 다시 확인해보자.

* Bag of Words, Binary ver
* Bag of Words, Count ver
* TF-IDF

In [0]:
print(X_simple[0,:])


In [0]:
print(X_count[0, :])

In [0]:
print(X_tfidf[0, :])

이제, tf-idf를 이용하여 nearest neighbor search 해보자!

In [0]:
from sklearn.neighbors import NearestNeighbors

In [0]:
nbrs_tfidf = NearestNeighbors(n_neighbors=6, metric='cosine').fit(X_tfidf)

안젤리나 졸리 비교 최종판!

In [0]:
nearest_search(name='Angelina Jolie', nbrs = nbrs_tfidf, X = X_tfidf)

In [0]:
nearest_search(name='Angelina Jolie', nbrs = nbrs_count, X = X_count)

In [0]:
nearest_search(name='Angelina Jolie', nbrs = nbrs_simple, X = X_simple)

빅토리아 베컴 비교 최종판!

In [0]:
nearest_search(name='Victoria Beckham', nbrs = nbrs_tfidf, X = X_tfidf)

In [0]:
nearest_search(name='Victoria Beckham', nbrs = nbrs_count, X = X_count)

In [0]:
nearest_search(name='Victoria Beckham', nbrs = nbrs_simple, X = X_simple)

## 클러스터링도 해봐야지!

DBSCAN을 해보자!

그리고, 감상하자.

메모리가 터지는 것을!

In [0]:
'''
라이브러리 로딩
'''

from sklearn.cluster import DBSCAN
from sklearn import metrics

In [0]:
dbscan_simple = DBSCAN(eps = 0.4, min_samples = 4, metric = 'cosine').fit(X_simple)

In [0]:
dbscan_simple.labels_